In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stat
import pylab 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
pd.set_option('display.max_columns', None)

df = pd.read_csv("C:/Project/Electricity_project/RawData/GlobalEYUtility_Final_for Demo.csv")
df = df[:300]
df.shape

(300, 119)

In [3]:
df.head()

,abin,Feeder_Category,BREAKER_MAKE,UpcaseRoadTraffic,Upcase_DepthOfCable,Upcase_SoilCondition,Upcase_LeadExposed,Upcase_ArmourCableCondition,CABLE_SIZE,NO__OF_PARALLEL_RUNNING_CABLES,FaultCurrent_Count,_5Yr_Faults_Count,No_of_Load_transfer_Operations,No_of_Times_Exceeded100_,No_of_times_exceeded_60_,No_of_time_below_60_,No_of_Times_Exceeded90_,LENGTH_OF_CABLE_MTRS_,NO__OF_JOINTS,AGE_YRS_,Number_OF_Cables,PILC,XLPE,DATE_ONLY,DAY_OF_WEEK,WEEKEND_Y_N_,Average_Current,MORNING_AVG_CURRENT,AFTERNOON_AVG_CURRENT,EVENING_AVG_CURRENT,Peak_Current,Peak_Slab,Average_voltage,MORNING_AVG_VOLTAGE,AFTERNOON_AVG_VOLTAGE,EVENING_AVG_VOLTAGE,Peak_Voltage,target,Avg_Heat_Index,Morning_AVG_Heat_Index,Afternoon_AVG_Heat_Index,Evening_Avg_Heat_Index,Peak_HI,Avg_humidity,Morning_AVG_humidity,Afternoon_AVG_HUMIDITY,Evening_Avg_HUMIDITY,Max_Humidity,Avg_TEMP,Morning_AVG_TEMP,Afternoon_AVG_TEMP,Evening_Avg_TEMP,Max_TEMP,IR_Value_After_JointingPHASE_TO,MORNING_AVG_CURRENT_N_1,MORNING_AVG_CURRENT_N_2,AFTERNOON_AVG_CURRENT_N_1,AFTERNOON_AVG_CURRENT_N_2,EVENING_AVG_CURRENT_N_1,EVENING_AVG_CURRENT_N_2,MORNING_AVG_VOLTAGE_N_1,MORNING_AVG_VOLTAGE_N_2,AFTERNOON_AVG_VOLTAGE_N_1,AFTERNOON_AVG_VOLTAGE_N_2,EVENING_AVG_VOLTAGE_N_1,EVENING_AVG_VOLTAGE_N_2,TARGET_N_1,TARGET_N_2,AVERAGE_CURRENT_N_1,AVERAGE_CURRENT_N_2,AVERAGE_VOLTAGE_N_1,AVERAGE_VOLTAGE_N_2,AVERAGE_HEAT_INDEX_N_1,AVERAGE_HEAT_INDEX_N_2,AVERAGE_HUMUDITY_N_1,AVERAGE_HUMUDITY_N_2,AVERAGE_TEMP_N_1,AVERAGE_TEMP_N_2,MORNING_AVG_HEAT_INDEX_N_1,MORNING_AVG_HEAT_INDEX_N_2,AFTERNOON_AVG_HEAT_INDEX_N_1,AFTERNOON_AVG_HEAT_INDEX_N_2,EVENING_AVG_HEAT_INDEX_N_1,EVENING_AVG_HEAT_INDEX_N_2,MORNING_AVG_HUMUDITY_N_1,MORNING_AVG_HUMUDITY_N_2,AFTERNOON_AVG_HUMUDITY_N_1,AFTERNOON_AVG_HUMUDITY_N_2,EVENING_AVG_HUMUDITY_N_1,EVENING_AVG_HUMUDITY_N_2,MORNING_AVG_TEMP_N_1,MORNING_AVG_TEMP_N_2,AFTERNOON_AVG_TEMP_N_1,AFTERNOON_AVG_TEMP_N_2,EVENING_AVG_TEMP_N_1,EVENING_AVG_TEMP_N_2,Peak_Current_N_1,Peak_Current_N_2,Peak_Current_Time_N_1,Peak_Current_Time_N_2,Peak_Current_Slab_N_1,Peak_Current_Slab_N_2,New_PublicHoliday,New_Summer,New_Monsoon,New_Winter,_dataobs_,Feeder,Switch,Section Id,Pathid,Path Order,Lat,Long,Customer At Risk,Revenue At Rsik,Assets At Risk,Risk Type,NoFaultsSince2016
0,ED,RES,SIEMENS VCB,MODERATE,NORMAL,DRY,NO,STRONG,120.0,3.0,2.0,5.0,0.0,0.0,0.0,96.0,0.0,586.6,7.0,36.0,2.0,1.0,1.0,08-01-2016,6.0,N,4.352917,9.688485,1.55125,1.565161,43.99,M,11.047187,11.035455,11.026250,11.081290,11.20,1.0,26.667515,23.993545,28.256467,27.873791,31.224829,60.078646,70.946061,48.330312,60.637419,79.34,25.887917,22.996970,27.952188,26.834516,31.11,250.0,68.730303,19.471818,108.005000,20.005000,107.405161,21.148710,11.035455,10.961818,11.026250,11.001875,11.081290,10.954839,0.0,0.0,94.310625,20.191042,11.047187,10.972917,26.667515,26.667515,60.078646,60.078646,25.887917,25.887917,23.993545,23.993545,28.256467,28.256467,27.873791,27.873791,70.946061,70.946061,48.330312,48.330312,60.637419,60.637419,22.996970,22.996970,27.952188,27.952188,26.834516,26.834516,1.5,9.08,0.0,0.0,M,M,N,N,N,Y,379.0,30462.0,10499.0,30462-10499-1,SANTACRUZ RECEIVING STATION-ANAND NAGAR NO.2 S...,2.0,19.080542,72.845893,8000.0,97.0,10.0,Low,0.0
1,ED,RES,ABB SF6,HEAVY,NORMAL,MOIST,NO,STRONG,300.0,1.0,7.0,2.0,0.0,0.0,0.0,96.0,0.0,533.1,5.0,NaN,1.0,0.0,1.0,15-01-2016,6.0,N,4.416875,12.658182,0.10000,0.100000,19.24,M,11.021667,10.977576,11.050000,11.039355,11.27,1.0,24.975674,22.618742,26.488964,25.922561,28.202562,69.590937,74.704545,63.073125,70.875484,76.92,23.927083,21.308182,25.579687,25.009032,27.30,350.0,58.424242,34.783333,79.528125,53.851250,88.881613,52.395484,10.977576,10.977576,11.050000,11.050000,11.039355,11.039355,0.0,0.0,75.294062,46.826563,11.021667,11.021667,24.975674,24.975674,69.590937,69.590937,23.927083,23.927083,22.618742,22.618742,26.488964,26.488964,25.922561,25.922561,74.704545,74.704545,63.073125,63.073125,70.875484,70.875484,21.308182,21.308182,25.579687,25.579687,25.009032,25.009032,0.1,18.46,0.0,0.0,M,M,N,N,N,Y,705.0,30462.0,10499.0

In [4]:
df = df[['Feeder_Category','UpcaseRoadTraffic','Upcase_SoilCondition','Upcase_ArmourCableCondition',
           'CABLE_SIZE','LENGTH_OF_CABLE_MTRS_','AGE_YRS_',
           'Average_Current','Average_voltage','Avg_Heat_Index','Avg_humidity','Avg_TEMP',
           'target']]
df

,Feeder_Category,UpcaseRoadTraffic,Upcase_SoilCondition,Upcase_ArmourCableCondition,CABLE_SIZE,LENGTH_OF_CABLE_MTRS_,AGE_YRS_,Average_Current,Average_voltage,Avg_Heat_Index,Avg_humidity,Avg_TEMP,target
0,RES,MODERATE,DRY,STRONG,120.0,586.6,36.0,4.352917,11.047187,26.667515,60.078646,25.887917,1.0
1,RES,HEAVY,MOIST,STRONG,300.0,533.1,NaN,4.416875,11.021667,24.975674,69.590937,23.927083,1.0
2,RES,MODERATE,DRY,STRONG,300.0,600.0,NaN,14.331250,11.066146,25.394108,62.813750,23.755521,1.0
3,RES,LOW,WET,CORRODED,240.0,682.5,23.0,27.535312,11.221042,25.914466,50.201458,24.734896,1.0
4,RES,MODERATE,DRY,STRONG,240.0,381.0,20.0,0.845625,11.077917,26.553766,64.328646,25.372917,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,COM,HEAVY,MOIST,CORRODED,240.0,1276.3,22.0,63.178125,11.044375,34.500309,74.840833,29.382917,1.0
296,COM,HEAVY,MOIST,CORRODED,240.0,1276.3,22.0,60.295521,11.067500,34.330905,62.240208,30.371562,1.0
297,RES,MODERATE,DRY,STRONG,240.0,387.1,18.0,94.052188,11.092187,26.413268,62.986250,25.350833,0.0
298,RES,MODERATE,DRY,CORRODED,240.0,1078.1,24.0,71.319583,11.070833,36.264426,81.260104,29.496875,1.0


In [5]:
df.describe(include='all')

,Feeder_Category,UpcaseRoadTraffic,Upcase_SoilCondition,Upcase_ArmourCableCondition,CABLE_SIZE,LENGTH_OF_CABLE_MTRS_,AGE_YRS_,Average_Current,Average_voltage,Avg_Heat_Index,Avg_humidity,Avg_TEMP,target
count,283,300,300,300,300.000000,300.000000,253.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
unique,3,3,4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,RES,HEAVY,DRY,STRONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,166,224,191,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,233.956833,1035.022703,22.750988,47.027596,10.720725,32.888793,76.306048,28.267487,0.580000
std,NaN,NaN,NaN,NaN,67.976865,584.411411,12.498780,41.829359,1.658994,5.083014,13.965015,2.031507,0.494383
min,NaN,NaN,NaN,NaN,0.150000,114.100000,3.000000,0.000000,0.000000,24.585061,47.395313,22.242812,0.000000
25%,NaN,NaN,NaN,NaN,240.000000,585.775000,12.000000,12.548281,10.985937,28.522308,64.549688,26.849010,0.000000
50%,NaN,NaN,NaN,NaN,240.000000,901.300000,20.000000,36.659010,11.034635,32.629315,76.432396,28.100729,1.000000
75%,NaN,NaN,NaN,NaN,300.000000,1346.482500,31.000000,70.110495,11.074974,36.057564,86.711172,29.946146,1.000000


In [6]:
df['Feeder_Category'] = df['Feeder_Category'].map({
                                                    'RES':0,
                                                    'COM':1,
                                                    'MIX':2})

df['Feeder_Category'].fillna(0,inplace=True)

df['UpcaseRoadTraffic'] = df['UpcaseRoadTraffic'].map({
    'HEAVY':0,
    'MODERATE':1,
    'LOW':2
})

df['UpcaseRoadTraffic'].fillna(0,inplace=True)

df['Upcase_SoilCondition'] = df['Upcase_SoilCondition'].map({
    'DRY':0,
    'MOIST':1,
    'WET':2,
    'ROCKY':3})

df['UpcaseRoadTraffic'].fillna(0,inplace=True)

df['Upcase_ArmourCableCondition'] = df['Upcase_ArmourCableCondition'].map({
    'STRONG':0,
    'CORRODED':1,
    'EXPOSED':2})

df['Upcase_ArmourCableCondition'].fillna(0,inplace=True)

df['CABLE_SIZE'] = df['CABLE_SIZE'].map({
    240.00 : 240.00,
    300.00 : 300.00,
    225.00 : 225.00,
    120.00: 120.00,
    185.00 : 120.00,
    70.00 : 120.00,
    0.15 : 0.15,
    0.20: 0.15,
    0.30 : 0.15
})

df['CABLE_SIZE'].fillna(240.00,inplace=True)

In [7]:
df['LENGTH_OF_CABLE_MTRS_'].fillna(df['LENGTH_OF_CABLE_MTRS_'].mean(),inplace=True)
df['AGE_YRS_'] = df['AGE_YRS_'].fillna(df['AGE_YRS_'].mean())

df['Average_Current']=df['Average_Current'].fillna(df['Average_Current'].mean())

In [49]:
df

,Feeder_Category,UpcaseRoadTraffic,Upcase_SoilCondition,Upcase_ArmourCableCondition,CABLE_SIZE,LENGTH_OF_CABLE_MTRS_,AGE_YRS_,Average_Current,Average_voltage,Avg_Heat_Index,Avg_humidity,Avg_TEMP,NO__OF_PARALLEL_RUNNING_CABLES,FaultCurrent_Count,No_of_times_exceeded_60_,NO__OF_JOINTS,Number_OF_Cables,MORNING_AVG_CURRENT,AFTERNOON_AVG_CURRENT,EVENING_AVG_CURRENT,Peak_Current,MORNING_AVG_VOLTAGE,AFTERNOON_AVG_VOLTAGE,EVENING_AVG_VOLTAGE,Peak_Voltage,Morning_AVG_Heat_Index,Afternoon_AVG_Heat_Index,Evening_Avg_Heat_Index,Peak_HI,Morning_AVG_humidity,Afternoon_AVG_HUMIDITY,Evening_Avg_HUMIDITY,Max_Humidity,Morning_AVG_TEMP,Afternoon_AVG_TEMP,Evening_Avg_TEMP,Max_TEMP,target
0,0.0,1,0,0,120.0,586.6,36.000000,4.352917,11.047187,26.667515,60.078646,25.887917,3.0,2.0,0.0,7.0,2.0,9.688485,1.551250,1.565161,43.99,11.035455,11.026250,11.081290,11.20,23.993545,28.256467,27.873791,31.224829,70.946061,48.330312,60.637419,79.34,22.996970,27.952188,26.834516,31.11,1.0
1,0.0,0,1,0,300.0,533.1,22.750988,4.416875,11.021667,24.975674,69.590937,23.927083,1.0,7.0,0.0,5.0,1.0,12.658182,0.100000,0.100000,19.24,10.977576,11.050000,11.039355,11.27,22.618742,26.488964,25.922561,28.202562,74.704545,63.073125,70.875484,76.92,21.308182,25.579687,25.009032,27.30,1.0
2,0.0,1,0,0,300.0,600.0,22.750988,14.331250,11.066146,25.394108,62.813750,23.755521,6.0,5.0,0.0,2.0,1.0,20.800000,20.800000,0.767742,20.80,11.131818,11.065312,10.997097,11.49,23.238414,26.747914,26.291401,29.479176,69.955758,56.565312,61.660968,73.11,20.099697,25.985937,25.344839,29.36,1.0
3,0.0,2,2,1,240.0,682.5,23.000000,27.535312,11.221042,25.914466,50.201458,24.734896,2.0,5.0,0.0,5.0,1.0,75.023333,5.141250,0.100000,109.47,11.276970,11.167813,11.216452,11.41,24.431373,27.060845,26.309882,28.942341,60.537879,39.792812,49.942581,64.66,21.665455,27.047500,25.615161,30.33,1.0
4,0.0,1,0,0,240.0,381.0,20.000000,0.845625,11.077917,26.553766,64.328646,25.372917,1.0,2.0,0.0,4.0,1.0,2.440909,0.010000,0.010000,80.23,11.085455,11.102813,11.044194,11.39,23.406862,29.567852,26.792381,32.863735,72.218485,55.434375,65.110968,74.42,22.035152,28.292813,25.911935,31.12,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,0,1,1,240.0,1276.3,22.000000,63.178125,11.044375,34.500309,74.840833,29.382917,8.0,6.0,0.0,6.0,1.0,58.000000,117.376250,12.743871,137.34,11.133636,11.004062,10.990968,11.31,30.190689,39.219166,34.216892,41.861295,80.446061,67.135000,76.828387,83.34,27.187273,31.969375,29.050323,33.50,1.0
296,1.0,0,1,1,240.0,1276.3,22.000000,60.295521,11.067500,34.330905,62.240208,30.371562,8.0,6.0,0.0,6.0,1.0,45.773333,110.946562,23.469677,131.87,11.157879,11.023750,11.016452,11.28,27.327041,40.512309,35.405827,44.262999,70.889697,62.601875,52.659355,78.17,25.975455,33.005000,32.332903,35.04,1.0
297,0.0,1,0,0,240.0,387.1,18.000000,94.052188,11.092187,26.413268,62.986250,25.350833,3.0,5.0,0.0,0.0,2.0,77.493939,103.659688,101.761290,117.08,11.054848,11.101875,11.121935,11.31,23.454430,29.058950,26.831971,32.266390,71.911212,52.906562,63.890323,74.90,22.144848,28.103750,25.921935,30.98,0.0
298,0.0,1,0,1,240.0,1078.1,24.000000,71.319583,11.070833,36.264426,81.260104,29.496875,3.0,14.0,14.0,11.0,1.0,116.449697,93.860313,0.010000,200.56,11.000909,11.050313,11.166452,11.38,32.360276,40.148108,36.411496,44.114422,87.132121,75.495000,80.960323,92.33,27.712121,31.290937,29.544839,32.78,1.0


In [50]:
df['target'].value_counts()

target
1.0    174
0.0    126
Name: count, dtype: int64

In [9]:
X = df.drop(columns=['target'])
y = df['target']

In [48]:
Xvalue_counts

Feeder_Category  UpcaseRoadTraffic  Upcase_SoilCondition  Upcase_ArmourCableCondition  CABLE_SIZE  LENGTH_OF_CABLE_MTRS_  AGE_YRS_   Average_Current  Average_voltage  Avg_Heat_Index  Avg_humidity  Avg_TEMP   NO__OF_PARALLEL_RUNNING_CABLES  FaultCurrent_Count  No_of_times_exceeded_60_  NO__OF_JOINTS  Number_OF_Cables  MORNING_AVG_CURRENT  AFTERNOON_AVG_CURRENT  EVENING_AVG_CURRENT  Peak_Current  MORNING_AVG_VOLTAGE  AFTERNOON_AVG_VOLTAGE  EVENING_AVG_VOLTAGE  Peak_Voltage  Morning_AVG_Heat_Index  Afternoon_AVG_Heat_Index  Evening_Avg_Heat_Index  Peak_HI    Morning_AVG_humidity  Afternoon_AVG_HUMIDITY  Evening_Avg_HUMIDITY  Max_Humidity  Morning_AVG_TEMP  Afternoon_AVG_TEMP  Evening_Avg_TEMP  Max_TEMP
0.0              0                  2                     0                            240.00      1609.71                20.000000  48.683437        10.815833        33.583378       75.698437     29.158646  3.0                             16.0                14.0                      27.0 

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
lrg = LogisticRegression()
lrg.fit(X_train,y_train)

C:\Users\YR272YB\.virtualenvs\Global_Utility_05-sbsjTaLH\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [31]:
m = lrg.coef_
m = m.tolist()

feature_names = lrg.feature_names_in_

pd.DataFrame({
    "feature_names":feature_names,
    "va":m[0]
})

,feature_names,va
0,Feeder_Category,-0.010639
1,UpcaseRoadTraffic,0.026667
2,Upcase_SoilCondition,0.082055
3,Upcase_ArmourCableCondition,0.000939
4,CABLE_SIZE,0.014764
5,LENGTH_OF_CABLE_MTRS_,-0.002207
6,AGE_YRS_,0.049577
7,Average_Current,-0.044195
8,Average_voltage,-0.030910
9,Avg_Heat_Index,0.039732


In [32]:
np.exp(lrg.coef_)

array([[0.98941702, 1.02702552, 1.08551503, 1.00093988, 1.01487306,
        0.997795  , 1.05082657, 0.95676748, 0.96956268, 1.04053153,
        0.99306533, 0.95722903, 0.99714999, 1.01436009, 1.09460898,
        1.07485167, 1.02164253, 1.08809559, 0.96420417, 0.82769578,
        1.11457825, 0.97620125, 0.98156218, 0.95040194, 0.9777632 ,
        1.06232804, 1.02736526, 1.03128748, 1.07055531, 0.93736454,
        1.02593203, 1.02109835, 0.99303137, 1.00061536, 0.93676494,
        0.93370679, 0.9406669 ]])

In [47]:


# create a sample dataframe
#df = pd.DataFrame({'exposed': [10, 20], 'unexposed': [20, 30]})

# calculate the 2x2 contingency table
contingency_table = pd.crosstab(X['Feeder_Category'],X['Max_TEMP'])

# calculate the odds of the outcome in each group
odds_exposed = contingency_table.iloc[1, 1] / contingency_table.iloc[1, 0]
odds_unexposed = contingency_table.iloc[0, 1] / contingency_table.iloc[0, 0]

# calculate the odds ratio
odds_ratio = odds_exposed / odds_unexposed

print('Odds ratio:', odds_ratio)


Odds ratio: nan


C:\Users\YR272YB\AppData\Local\Temp\ipykernel_22180\3734407553.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  odds_exposed = contingency_table.iloc[1, 1] / contingency_table.iloc[1, 0]
C:\Users\YR272YB\AppData\Local\Temp\ipykernel_22180\3734407553.py:11: RuntimeWarning: divide by zero encountered in longlong_scalars
  odds_unexposed = contingency_table.iloc[0, 1] / contingency_table.iloc[0, 0]


### Interpreting coefficients

* An unit change in Avg_TEMP decreases 0.336557 units, or almost 33%, of the prediction being faulty.
* An unit change in Avg_humidity decreases 0.040819 units, or almost 4%, of the prediction being faulty.
* An unit change in Avg_Heat_Index increases 0.326266 units, or almost 32%, of the prediction being faulty.
* An unit change in Average_voltage decreases 0.101450 units, or almost 10%, of the prediction being faulty.
* An unit change in Average_Current decreases 0.017486 units, or almost 1%, of the prediction being faulty.
* An unit change in AGE_YRS_ increases 0.046135 units, or almost 4%, of the prediction being faulty.

In [13]:
df

,Feeder_Category,UpcaseRoadTraffic,Upcase_SoilCondition,Upcase_ArmourCableCondition,CABLE_SIZE,LENGTH_OF_CABLE_MTRS_,AGE_YRS_,Average_Current,Average_voltage,Avg_Heat_Index,Avg_humidity,Avg_TEMP,target
0,0.0,1,0,0,120.0,586.6,36.000000,4.352917,11.047187,26.667515,60.078646,25.887917,1.0
1,0.0,0,1,0,300.0,533.1,22.750988,4.416875,11.021667,24.975674,69.590937,23.927083,1.0
2,0.0,1,0,0,300.0,600.0,22.750988,14.331250,11.066146,25.394108,62.813750,23.755521,1.0
3,0.0,2,2,1,240.0,682.5,23.000000,27.535312,11.221042,25.914466,50.201458,24.734896,1.0
4,0.0,1,0,0,240.0,381.0,20.000000,0.845625,11.077917,26.553766,64.328646,25.372917,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,0,1,1,240.0,1276.3,22.000000,63.178125,11.044375,34.500309,74.840833,29.382917,1.0
296,1.0,0,1,1,240.0,1276.3,22.000000,60.295521,11.067500,34.330905,62.240208,30.371562,1.0
297,0.0,1,0,0,240.0,387.1,18.000000,94.052188,11.092187,26.413268,62.986250,25.350833,0.0
298,0.0,1,0,1,240.0,1078.1,24.000000,71.319583,11.070833,36.264426,81.260104,29.496875,1.0


### Considering all features

In [14]:
pd.set_option('display.max_columns', None)

df = pd.read_csv("C:/Project/Electricity_project/RawData/GlobalEYUtility_Final_for Demo.csv")
df = df[:300]
df.shape
df.head()

,abin,Feeder_Category,BREAKER_MAKE,UpcaseRoadTraffic,Upcase_DepthOfCable,Upcase_SoilCondition,Upcase_LeadExposed,Upcase_ArmourCableCondition,CABLE_SIZE,NO__OF_PARALLEL_RUNNING_CABLES,FaultCurrent_Count,_5Yr_Faults_Count,No_of_Load_transfer_Operations,No_of_Times_Exceeded100_,No_of_times_exceeded_60_,No_of_time_below_60_,No_of_Times_Exceeded90_,LENGTH_OF_CABLE_MTRS_,NO__OF_JOINTS,AGE_YRS_,Number_OF_Cables,PILC,XLPE,DATE_ONLY,DAY_OF_WEEK,WEEKEND_Y_N_,Average_Current,MORNING_AVG_CURRENT,AFTERNOON_AVG_CURRENT,EVENING_AVG_CURRENT,Peak_Current,Peak_Slab,Average_voltage,MORNING_AVG_VOLTAGE,AFTERNOON_AVG_VOLTAGE,EVENING_AVG_VOLTAGE,Peak_Voltage,target,Avg_Heat_Index,Morning_AVG_Heat_Index,Afternoon_AVG_Heat_Index,Evening_Avg_Heat_Index,Peak_HI,Avg_humidity,Morning_AVG_humidity,Afternoon_AVG_HUMIDITY,Evening_Avg_HUMIDITY,Max_Humidity,Avg_TEMP,Morning_AVG_TEMP,Afternoon_AVG_TEMP,Evening_Avg_TEMP,Max_TEMP,IR_Value_After_JointingPHASE_TO,MORNING_AVG_CURRENT_N_1,MORNING_AVG_CURRENT_N_2,AFTERNOON_AVG_CURRENT_N_1,AFTERNOON_AVG_CURRENT_N_2,EVENING_AVG_CURRENT_N_1,EVENING_AVG_CURRENT_N_2,MORNING_AVG_VOLTAGE_N_1,MORNING_AVG_VOLTAGE_N_2,AFTERNOON_AVG_VOLTAGE_N_1,AFTERNOON_AVG_VOLTAGE_N_2,EVENING_AVG_VOLTAGE_N_1,EVENING_AVG_VOLTAGE_N_2,TARGET_N_1,TARGET_N_2,AVERAGE_CURRENT_N_1,AVERAGE_CURRENT_N_2,AVERAGE_VOLTAGE_N_1,AVERAGE_VOLTAGE_N_2,AVERAGE_HEAT_INDEX_N_1,AVERAGE_HEAT_INDEX_N_2,AVERAGE_HUMUDITY_N_1,AVERAGE_HUMUDITY_N_2,AVERAGE_TEMP_N_1,AVERAGE_TEMP_N_2,MORNING_AVG_HEAT_INDEX_N_1,MORNING_AVG_HEAT_INDEX_N_2,AFTERNOON_AVG_HEAT_INDEX_N_1,AFTERNOON_AVG_HEAT_INDEX_N_2,EVENING_AVG_HEAT_INDEX_N_1,EVENING_AVG_HEAT_INDEX_N_2,MORNING_AVG_HUMUDITY_N_1,MORNING_AVG_HUMUDITY_N_2,AFTERNOON_AVG_HUMUDITY_N_1,AFTERNOON_AVG_HUMUDITY_N_2,EVENING_AVG_HUMUDITY_N_1,EVENING_AVG_HUMUDITY_N_2,MORNING_AVG_TEMP_N_1,MORNING_AVG_TEMP_N_2,AFTERNOON_AVG_TEMP_N_1,AFTERNOON_AVG_TEMP_N_2,EVENING_AVG_TEMP_N_1,EVENING_AVG_TEMP_N_2,Peak_Current_N_1,Peak_Current_N_2,Peak_Current_Time_N_1,Peak_Current_Time_N_2,Peak_Current_Slab_N_1,Peak_Current_Slab_N_2,New_PublicHoliday,New_Summer,New_Monsoon,New_Winter,_dataobs_,Feeder,Switch,Section Id,Pathid,Path Order,Lat,Long,Customer At Risk,Revenue At Rsik,Assets At Risk,Risk Type,NoFaultsSince2016
0,ED,RES,SIEMENS VCB,MODERATE,NORMAL,DRY,NO,STRONG,120.0,3.0,2.0,5.0,0.0,0.0,0.0,96.0,0.0,586.6,7.0,36.0,2.0,1.0,1.0,08-01-2016,6.0,N,4.352917,9.688485,1.55125,1.565161,43.99,M,11.047187,11.035455,11.026250,11.081290,11.20,1.0,26.667515,23.993545,28.256467,27.873791,31.224829,60.078646,70.946061,48.330312,60.637419,79.34,25.887917,22.996970,27.952188,26.834516,31.11,250.0,68.730303,19.471818,108.005000,20.005000,107.405161,21.148710,11.035455,10.961818,11.026250,11.001875,11.081290,10.954839,0.0,0.0,94.310625,20.191042,11.047187,10.972917,26.667515,26.667515,60.078646,60.078646,25.887917,25.887917,23.993545,23.993545,28.256467,28.256467,27.873791,27.873791,70.946061,70.946061,48.330312,48.330312,60.637419,60.637419,22.996970,22.996970,27.952188,27.952188,26.834516,26.834516,1.5,9.08,0.0,0.0,M,M,N,N,N,Y,379.0,30462.0,10499.0,30462-10499-1,SANTACRUZ RECEIVING STATION-ANAND NAGAR NO.2 S...,2.0,19.080542,72.845893,8000.0,97.0,10.0,Low,0.0
1,ED,RES,ABB SF6,HEAVY,NORMAL,MOIST,NO,STRONG,300.0,1.0,7.0,2.0,0.0,0.0,0.0,96.0,0.0,533.1,5.0,NaN,1.0,0.0,1.0,15-01-2016,6.0,N,4.416875,12.658182,0.10000,0.100000,19.24,M,11.021667,10.977576,11.050000,11.039355,11.27,1.0,24.975674,22.618742,26.488964,25.922561,28.202562,69.590937,74.704545,63.073125,70.875484,76.92,23.927083,21.308182,25.579687,25.009032,27.30,350.0,58.424242,34.783333,79.528125,53.851250,88.881613,52.395484,10.977576,10.977576,11.050000,11.050000,11.039355,11.039355,0.0,0.0,75.294062,46.826563,11.021667,11.021667,24.975674,24.975674,69.590937,69.590937,23.927083,23.927083,22.618742,22.618742,26.488964,26.488964,25.922561,25.922561,74.704545,74.704545,63.073125,63.073125,70.875484,70.875484,21.308182,21.308182,25.579687,25.579687,25.009032,25.009032,0.1,18.46,0.0,0.0,M,M,N,N,N,Y,705.0,30462.0,10499.0

In [15]:
df['Feeder_Category'] = df['Feeder_Category'].map({
                                                    'RES':0,
                                                    'COM':1,
                                                    'MIX':2})

df['Feeder_Category'].fillna(0,inplace=True)

df['UpcaseRoadTraffic'] = df['UpcaseRoadTraffic'].map({
    'HEAVY':0,
    'MODERATE':1,
    'LOW':2
})

df['UpcaseRoadTraffic'].fillna(0,inplace=True)

df['Upcase_SoilCondition'] = df['Upcase_SoilCondition'].map({
    'DRY':0,
    'MOIST':1,
    'WET':2,
    'ROCKY':3})

df['UpcaseRoadTraffic'].fillna(0,inplace=True)

df['Upcase_ArmourCableCondition'] = df['Upcase_ArmourCableCondition'].map({
    'STRONG':0,
    'CORRODED':1,
    'EXPOSED':2})

df['Upcase_ArmourCableCondition'].fillna(0,inplace=True)

df['CABLE_SIZE'] = df['CABLE_SIZE'].map({
    240.00 : 240.00,
    300.00 : 300.00,
    225.00 : 225.00,
    120.00: 120.00,
    185.00 : 120.00,
    70.00 : 120.00,
    0.15 : 0.15,
    0.20: 0.15,
    0.30 : 0.15
})

df['CABLE_SIZE'].fillna(240.00,inplace=True)

In [16]:
df['LENGTH_OF_CABLE_MTRS_'].fillna(df['LENGTH_OF_CABLE_MTRS_'].mean(),inplace=True)
df['AGE_YRS_'] = df['AGE_YRS_'].fillna(df['AGE_YRS_'].mean())

df['Average_Current']=df['Average_Current'].fillna(df['Average_Current'].mean())
df['NO__OF_PARALLEL_RUNNING_CABLES'] = df['NO__OF_PARALLEL_RUNNING_CABLES'].fillna(df['NO__OF_PARALLEL_RUNNING_CABLES'].mean())
df['FaultCurrent_Count'] = df['FaultCurrent_Count'].fillna(df['FaultCurrent_Count'].mean())
df['No_of_times_exceeded_60_'] = df['No_of_times_exceeded_60_'].fillna(df['No_of_times_exceeded_60_'].mean())
df['NO__OF_JOINTS'] = df['NO__OF_JOINTS'].fillna(df['NO__OF_JOINTS'].mean())
df['Number_OF_Cables'] = df['Number_OF_Cables'].fillna(df['NO__OF_JOINTS'].mean())
df['MORNING_AVG_CURRENT'] = df['MORNING_AVG_CURRENT'].fillna(df['MORNING_AVG_CURRENT'].mean())
df['AFTERNOON_AVG_CURRENT'] = df['AFTERNOON_AVG_CURRENT'].fillna(df['AFTERNOON_AVG_CURRENT'].mean())
df['EVENING_AVG_CURRENT'] = df['EVENING_AVG_CURRENT'].fillna(df['EVENING_AVG_CURRENT'].mean())
df['Peak_Current'] = df['Peak_Current'].fillna(df['Peak_Current'].mean())

df['Average_voltage'] = df['Average_voltage'].fillna(df['Average_voltage'].mean())
df['MORNING_AVG_VOLTAGE'] = df['MORNING_AVG_VOLTAGE'].fillna(df['MORNING_AVG_VOLTAGE'].mean())
df['AFTERNOON_AVG_VOLTAGE'] = df['AFTERNOON_AVG_VOLTAGE'].fillna(df['AFTERNOON_AVG_VOLTAGE'].mean())
df['EVENING_AVG_VOLTAGE'] = df['EVENING_AVG_VOLTAGE'].fillna(df['EVENING_AVG_VOLTAGE'].mean())
df['Peak_Voltage'] = df['Peak_Voltage'].fillna(df['Peak_Voltage'].mean())

df['Avg_Heat_Index'] = df['Avg_Heat_Index'].fillna(df['Avg_Heat_Index'].mean())
df['Morning_AVG_Heat_Index'] = df['Morning_AVG_Heat_Index'].fillna(df['Morning_AVG_Heat_Index'].mean())
df['Afternoon_AVG_Heat_Index'] = df['Afternoon_AVG_Heat_Index'].fillna(df['Afternoon_AVG_Heat_Index'].mean())
df['Evening_Avg_Heat_Index'] = df['Evening_Avg_Heat_Index'].fillna(df['Evening_Avg_Heat_Index'].mean())
df['Peak_HI'] = df['Peak_HI'].fillna(df['Peak_HI'].mean())

df['Avg_humidity'] = df['Avg_humidity'].fillna(df['Avg_humidity'].mean())
df['Morning_AVG_humidity'] = df['Morning_AVG_humidity'].fillna(df['Morning_AVG_humidity'].mean())
df['Afternoon_AVG_HUMIDITY'] = df['Afternoon_AVG_HUMIDITY'].fillna(df['Afternoon_AVG_HUMIDITY'].mean())
df['Evening_Avg_HUMIDITY'] = df['Evening_Avg_HUMIDITY'].fillna(df['Evening_Avg_HUMIDITY'].mean())
df['Max_Humidity'] = df['Max_Humidity'].fillna(df['Max_Humidity'].mean())

df['Avg_TEMP'] = df['Avg_TEMP'].fillna(df['Avg_TEMP'].mean())
df['Morning_AVG_TEMP'] = df['Morning_AVG_TEMP'].fillna(df['Morning_AVG_TEMP'].mean())
df['Afternoon_AVG_TEMP'] = df['Afternoon_AVG_TEMP'].fillna(df['Afternoon_AVG_TEMP'].mean())
df['Evening_Avg_TEMP'] = df['Evening_Avg_TEMP'].fillna(df['Evening_Avg_TEMP'].mean())
df['Max_TEMP'] = df['Max_TEMP'].fillna(df['Max_TEMP'].mean())

In [17]:
df = df[['Feeder_Category','UpcaseRoadTraffic','Upcase_SoilCondition','Upcase_ArmourCableCondition',
           'CABLE_SIZE','LENGTH_OF_CABLE_MTRS_','AGE_YRS_',
           'Average_Current','Average_voltage','Avg_Heat_Index','Avg_humidity','Avg_TEMP','NO__OF_PARALLEL_RUNNING_CABLES','FaultCurrent_Count',
           'No_of_times_exceeded_60_','NO__OF_JOINTS','Number_OF_Cables','MORNING_AVG_CURRENT','AFTERNOON_AVG_CURRENT','EVENING_AVG_CURRENT','Peak_Current',
           'MORNING_AVG_VOLTAGE','AFTERNOON_AVG_VOLTAGE','EVENING_AVG_VOLTAGE','Peak_Voltage','Morning_AVG_Heat_Index','Afternoon_AVG_Heat_Index','Evening_Avg_Heat_Index',
           'Peak_HI','Morning_AVG_humidity','Afternoon_AVG_HUMIDITY','Evening_Avg_HUMIDITY','Max_Humidity','Morning_AVG_TEMP','Afternoon_AVG_TEMP',
           'Evening_Avg_TEMP','Max_TEMP',
           'target']]

df

,Feeder_Category,UpcaseRoadTraffic,Upcase_SoilCondition,Upcase_ArmourCableCondition,CABLE_SIZE,LENGTH_OF_CABLE_MTRS_,AGE_YRS_,Average_Current,Average_voltage,Avg_Heat_Index,Avg_humidity,Avg_TEMP,NO__OF_PARALLEL_RUNNING_CABLES,FaultCurrent_Count,No_of_times_exceeded_60_,NO__OF_JOINTS,Number_OF_Cables,MORNING_AVG_CURRENT,AFTERNOON_AVG_CURRENT,EVENING_AVG_CURRENT,Peak_Current,MORNING_AVG_VOLTAGE,AFTERNOON_AVG_VOLTAGE,EVENING_AVG_VOLTAGE,Peak_Voltage,Morning_AVG_Heat_Index,Afternoon_AVG_Heat_Index,Evening_Avg_Heat_Index,Peak_HI,Morning_AVG_humidity,Afternoon_AVG_HUMIDITY,Evening_Avg_HUMIDITY,Max_Humidity,Morning_AVG_TEMP,Afternoon_AVG_TEMP,Evening_Avg_TEMP,Max_TEMP,target
0,0.0,1,0,0,120.0,586.6,36.000000,4.352917,11.047187,26.667515,60.078646,25.887917,3.0,2.0,0.0,7.0,2.0,9.688485,1.551250,1.565161,43.99,11.035455,11.026250,11.081290,11.20,23.993545,28.256467,27.873791,31.224829,70.946061,48.330312,60.637419,79.34,22.996970,27.952188,26.834516,31.11,1.0
1,0.0,0,1,0,300.0,533.1,22.750988,4.416875,11.021667,24.975674,69.590937,23.927083,1.0,7.0,0.0,5.0,1.0,12.658182,0.100000,0.100000,19.24,10.977576,11.050000,11.039355,11.27,22.618742,26.488964,25.922561,28.202562,74.704545,63.073125,70.875484,76.92,21.308182,25.579687,25.009032,27.30,1.0
2,0.0,1,0,0,300.0,600.0,22.750988,14.331250,11.066146,25.394108,62.813750,23.755521,6.0,5.0,0.0,2.0,1.0,20.800000,20.800000,0.767742,20.80,11.131818,11.065312,10.997097,11.49,23.238414,26.747914,26.291401,29.479176,69.955758,56.565312,61.660968,73.11,20.099697,25.985937,25.344839,29.36,1.0
3,0.0,2,2,1,240.0,682.5,23.000000,27.535312,11.221042,25.914466,50.201458,24.734896,2.0,5.0,0.0,5.0,1.0,75.023333,5.141250,0.100000,109.47,11.276970,11.167813,11.216452,11.41,24.431373,27.060845,26.309882,28.942341,60.537879,39.792812,49.942581,64.66,21.665455,27.047500,25.615161,30.33,1.0
4,0.0,1,0,0,240.0,381.0,20.000000,0.845625,11.077917,26.553766,64.328646,25.372917,1.0,2.0,0.0,4.0,1.0,2.440909,0.010000,0.010000,80.23,11.085455,11.102813,11.044194,11.39,23.406862,29.567852,26.792381,32.863735,72.218485,55.434375,65.110968,74.42,22.035152,28.292813,25.911935,31.12,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,0,1,1,240.0,1276.3,22.000000,63.178125,11.044375,34.500309,74.840833,29.382917,8.0,6.0,0.0,6.0,1.0,58.000000,117.376250,12.743871,137.34,11.133636,11.004062,10.990968,11.31,30.190689,39.219166,34.216892,41.861295,80.446061,67.135000,76.828387,83.34,27.187273,31.969375,29.050323,33.50,1.0
296,1.0,0,1,1,240.0,1276.3,22.000000,60.295521,11.067500,34.330905,62.240208,30.371562,8.0,6.0,0.0,6.0,1.0,45.773333,110.946562,23.469677,131.87,11.157879,11.023750,11.016452,11.28,27.327041,40.512309,35.405827,44.262999,70.889697,62.601875,52.659355,78.17,25.975455,33.005000,32.332903,35.04,1.0
297,0.0,1,0,0,240.0,387.1,18.000000,94.052188,11.092187,26.413268,62.986250,25.350833,3.0,5.0,0.0,0.0,2.0,77.493939,103.659688,101.761290,117.08,11.054848,11.101875,11.121935,11.31,23.454430,29.058950,26.831971,32.266390,71.911212,52.906562,63.890323,74.90,22.144848,28.103750,25.921935,30.98,0.0
298,0.0,1,0,1,240.0,1078.1,24.000000,71.319583,11.070833,36.264426,81.260104,29.496875,3.0,14.0,14.0,11.0,1.0,116.449697,93.860313,0.010000,200.56,11.000909,11.050313,11.166452,11.38,32.360276,40.148108,36.411496,44.114422,87.132121,75.495000,80.960323,92.33,27.712121,31.290937,29.544839,32.78,1.0


In [18]:
X = df.drop(columns=['target'])
y = df['target']

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

lrg = LogisticRegression()
lrg.fit(X_train,y_train)

C:\Users\YR272YB\.virtualenvs\Global_Utility_05-sbsjTaLH\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
m = lrg.coef_
m = m.tolist()

feature_names = lrg.feature_names_in_

pd.DataFrame({
    "feature_names":feature_names,
    "va":m[0]
})

,feature_names,va
0,Feeder_Category,-0.010639
1,UpcaseRoadTraffic,0.026667
2,Upcase_SoilCondition,0.082055
3,Upcase_ArmourCableCondition,0.000939
4,CABLE_SIZE,0.014764
5,LENGTH_OF_CABLE_MTRS_,-0.002207
6,AGE_YRS_,0.049577
7,Average_Current,-0.044195
8,Average_voltage,-0.030910
9,Avg_Heat_Index,0.039732
